In [ ]:
import numpy as np
import pandas as pd
import torch
%pip install transformers # install transformers explicitly
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 778kB 4.6MB/s 
     |████████████████████████████████| 1.1MB 21.8MB/s 
     |████████████████████████████████| 3.0MB 38.7MB/s 
     |████████████████████████████████| 890kB 38.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=513ca320176f2d3d83f51eacb14ceecfaf94d3eab98471703c356fac168d52a8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import files
data_to_load = files.upload()

Saving train.csv to train.csv


In [ ]:
import io
#train = pd.read_csv(io.StringIO(data_to_load["train.csv"].decode("utf-8")))
train = pd.read_csv("train.csv")
train.head()

FileNotFoundError: ignored

In [ ]:
train['total'] = train['text']+train['title']+train['author']

In [ ]:
batch_1 = train[:200]

In [ ]:
batch_1['label'].value_counts()

0    103
1     97
Name: label, dtype: int64

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
clean_train_set = train.dropna()
batch_1 = clean_train_set[:200]


In [ ]:
tokenized = batch_1['total'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True)))

In [ ]:
tokenized

0      [101, 2160, 17183, 14895, 1024, 2057, 2134, 15...
1      [101, 2412, 2131, 1996, 3110, 2115, 2166, 7925...
2      [101, 2339, 1996, 3606, 2453, 2131, 2017, 5045...
3      [101, 6876, 2321, 9272, 2730, 1999, 2309, 2149...
4      [101, 6140, 2019, 7726, 2450, 2038, 2042, 7331...
                             ...                        
226    [101, 6140, 1999, 2262, 1010, 7206, 1999, 9899...
227    [101, 2003, 1996, 7349, 2012, 3891, 2005, 2613...
228    [101, 2255, 2654, 1010, 2355, 11176, 2096, 199...
230    [101, 6676, 4811, 1010, 1996, 3364, 2124, 2005...
232    [101, 2899, 1517, 1996, 2976, 3914, 2992, 2049...
Name: total, Length: 200, dtype: object

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
np.array(padded).shape

(200, 512)

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(200, 512)

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
last_hidden_states

(tensor([[[-0.2179, -0.2663, -0.2777,  ...,  0.0814,  0.3518,  0.3551],
          [ 0.4516,  0.2222,  0.2031,  ...,  0.2081,  0.0144,  0.4772],
          [-0.3720, -0.2645, -0.3346,  ..., -0.1060, -0.1355,  0.4169],
          ...,
          [ 0.0739,  0.7109, -0.2540,  ...,  0.1353, -0.3009, -0.5338],
          [ 0.2773,  0.2075,  0.0832,  ...,  0.1456,  0.0513, -0.7211],
          [-0.3099, -0.1577, -0.0397,  ...,  0.6019,  0.2351, -0.5509]],
 
         [[-0.1600, -0.2211, -0.1660,  ...,  0.0848,  0.3876,  0.4130],
          [ 0.1071,  0.0861,  0.5544,  ...,  0.2376,  0.5008, -0.0472],
          [-0.3878,  0.0677,  0.4318,  ..., -0.0424, -0.0048,  0.1756],
          ...,
          [-0.2836, -0.3645,  0.0552,  ..., -0.0546, -0.4330,  0.1342],
          [-0.3870, -0.1244,  0.1532,  ..., -0.1050, -0.2343, -0.3688],
          [-0.4441, -0.0956, -0.3927,  ...,  0.5128,  0.2798, -0.4220]],
 
         [[-0.1645, -0.1228, -0.5196,  ..., -0.0413,  0.5080,  0.4314],
          [-0.2607,  0.4052,

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = batch_1['label']

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression(max_iter = 3000)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=3000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr_clf.score(test_features, test_labels)

0.8

<h1> LSTM+BERT </h1>

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Dropout


In [ ]:
voc_size = 500
sent_leng = 768
embedded_docs = pad_sequences(last_hidden_states, padding = 'pre',maxlen = sent_leng)


In [ ]:
embedding_vector_features = 40    #model handles this part automatically
lstm_model = Sequential()
lstm_model.add(Embedding(voc_size, embedding_vector_features ,input_length = sent_leng))
lstm_model.add(Dropout(0.3))
lstm_model.add(LSTM(100))  #1 LSTM Layer with 100 neurons
lstm_model.add(Dense(1 , activation = 'sigmoid'))
lstm_model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
print(lstm_model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 768, 40)           20000     
_________________________________________________________________
dropout (Dropout)            (None, 768, 40)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 76,501
Trainable params: 76,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
labels = batch_1['label']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
len(train_features)
len(test_features)

50

In [ ]:
lstm_model.fit(train_features , train_labels , validation_data=(test_features,test_labels) , epochs= 10 , batch_size = 64)  

Epoch 1/10


InvalidArgumentError: ignored